In [17]:
import numpy as np 
import matplotlib as plt 
import os 
import time
import cv2 
import pandas as pd
import tensorflowjs as tfjs
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models,activations
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input,LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

## Load and Preprocess Data

In [26]:
DATA_PATH = os.path.DATA_PATH = os.path.join(os.getcwd(),"Fingerpose_data")
gestures = ['play','pause','nothing']
curl_map = {'No Curl':0,'Half Curl':1,'Full Curl':2}
direction_map = {'Vertical Up':0,'Vertical Down':1,'Horizontal Left':2,'Horizontal Right':3,'Diagonal Up Right':4,
                 'Diagonal Up Left':5,'Diagonal Down Right':6,'Diagonal Down Left':7}
label_map = {label:num for num, label in enumerate(gestures)}

In [27]:
#read data from CSV file and save to a dataset 
def load_csv(data,gesture,dataset,labels):
    data = data.reset_index()
    frame,datapoints = [],[]
    for index, row in data.iterrows():
        frame.append(curl_map[row['Curl']])
        frame.append(direction_map[row['Direction']])
        if (index+1)%5 == 0:
            datapoints.append(frame)
            frame = []
        if (index+1)%150 == 0:
            dataset.append(datapoints)
            labels.append(label_map[gesture])
            datapoints=[]

In [28]:
dataset,labels = [],[]
for gesture in gestures:
    files = os.listdir(os.path.join(DATA_PATH,gesture))
    for file in files:
        df = pd.read_csv(os.path.join(DATA_PATH,gesture,file))
        load_csv(df,gesture,dataset,labels)
        
            

In [29]:
X = np.array(dataset)
y = to_categorical(labels).astype(int)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

(60, 30, 10)
(60, 3)


## Train LSTM Model 

In [30]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [31]:
inputs = Input((30,10))
lstm = LSTM(64,return_sequences=True, activation = 'relu',input_shape=(30,10))(inputs)
lstm2 = LSTM(128, activation = 'relu')(lstm)
de = Dense(128, activation='relu')(lstm2)
de2 = Dense(64, activation='relu')(de)
outputs = Dense(len(gestures),activation='softmax')(de2)
model = models.Model(inputs,outputs)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 30, 10)]          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 64)            19200     
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 195       
Total params: 142,979
Trainable params: 142,979
Non-trainable params: 0
_____________________________________________________

In [32]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
2/2 [==============================] - 3s 1s/step - loss: 1.0926 - categorical_accuracy: 0.3889
Epoch 2/100
2/2 [==============================] - 0s 249ms/step - loss: 1.0657 - categorical_accuracy: 0.6481
Epoch 3/100
2/2 [==============================] - 0s 227ms/step - loss: 0.9744 - categorical_accuracy: 0.6852
Epoch 4/100
2/2 [==============================] - 0s 220ms/step - loss: 0.8738 - categorical_accuracy: 0.8519
Epoch 5/100
2/2 [==============================] - 0s 236ms/step - loss: 0.7135 - categorical_accuracy: 0.8519
Epoch 6/100
2/2 [==============================] - 1s 228ms/step - loss: 0.4893 - categorical_accuracy: 0.9259
Epoch 7/100
2/2 [==============================] - 0s 233ms/step - loss: 1.0465 - categorical_accuracy: 0.7222
Epoch 8/100
2/2 [==============================] - 0s 244ms/step - loss: 0.6191 - categorical_accuracy: 0.7593
Epoch 9/100
2/2 [==============================] - 1s 277ms/step - loss: 0.3704 - categorical_accuracy: 0.8889
Epoc

2/2 [==============================] - 0s 230ms/step - loss: 1.0243e-06 - categorical_accuracy: 1.0000
Epoch 74/100
2/2 [==============================] - 0s 222ms/step - loss: 9.9339e-07 - categorical_accuracy: 1.0000
Epoch 75/100
2/2 [==============================] - 0s 220ms/step - loss: 9.1612e-07 - categorical_accuracy: 1.0000
Epoch 76/100
2/2 [==============================] - 0s 255ms/step - loss: 8.6977e-07 - categorical_accuracy: 1.0000
Epoch 77/100
2/2 [==============================] - 0s 241ms/step - loss: 8.3003e-07 - categorical_accuracy: 1.0000
Epoch 78/100
2/2 [==============================] - 0s 219ms/step - loss: 8.1017e-07 - categorical_accuracy: 1.0000
Epoch 79/100
2/2 [==============================] - 0s 228ms/step - loss: 7.5277e-07 - categorical_accuracy: 1.0000
Epoch 80/100
2/2 [==============================] - 0s 232ms/step - loss: 7.3953e-07 - categorical_accuracy: 1.0000
Epoch 81/100
2/2 [==============================] - 0s 218ms/step - loss: 6.9758e-07 

In [33]:
#convert and save file in tfjs format 
tfjs.converters.save_keras_model(model,os.getcwd())

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
#save model in python format
model.save('gestures.h5')